## Carregando as Bases de dados

In [21]:
# imports

import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


In [22]:
# FUNÇÃO PARA VERIFICAR A QUANTIDADE DE DADOS FALTANTES

def missing_data_info(data):
    missing_count = data.isnull().sum()
    missing_percent = 100 * data.isnull().mean()
    data_types = data.dtypes
    missing_info = pd.DataFrame({
        'Missing Count': missing_count,
        'Missing Percent': missing_percent,
        'Data Type': data_types
    })

    return missing_info.sort_values(by='Missing Percent', ascending=False)

# missing_data_info(df)

## Preparação da base

In [23]:
# Carregando o dataset FAMILIAS/pessoas

pd.set_option('display.max_columns', 51)
pd.set_option('display.max_rows', 51)

df = pd.read_csv('bases/df_fam_pessoas_amostra_100mil.csv', sep=',')
print(f'Shape:', df.shape)

df.head()

Shape: (267140, 51)


,cd_ibge,uf_ibge,regiao_ibge,estrato,classf,id_familia,vlr_renda_media_fam,cod_local_domic_fam,cod_especie_domic_fam,qtd_comodos_domic_fam,qtd_comodos_dormitorio_fam,cod_material_piso_fam,cod_material_domic_fam,cod_agua_canalizada_fam,cod_abaste_agua_domic_fam,cod_banheiro_domic_fam,cod_escoa_sanitario_domic_fam,cod_destino_lixo_domic_fam,cod_iluminacao_domic_fam,cod_calcamento_domic_fam,cod_familia_indigena_fam,ind_familia_quilombola_fam,ind_parc_mds_fam,qtde_pessoas,peso.fam,dias_cadastramento,dias_atualizacao,classe_renda,id_pessoa,cod_sexo_pessoa,idade,cod_parentesco_rf_pessoa,cod_raca_cor_pessoa,cod_local_nascimento_pessoa,cod_certidao_registrada_pessoa,cod_deficiencia_memb,cod_sabe_ler_escrever_memb,ind_frequenta_escola_memb,cod_escola_local_memb,cod_curso_frequenta_memb,cod_ano_serie_frequenta_memb,cod_curso_frequentou_pessoa_memb,cod_ano_serie_frequentou_memb,cod_concluiu_frequentou_memb,cod_trabalhou_memb,cod_afastado_trab_memb,cod_agricultura_trab_memb,cod_principal_trab_memb,cod_trabalho_12_meses_memb,qtd_meses_12_meses_memb,peso.pes
0,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,2351897,2,4,3,1.0,2.0,1,2,2.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563277716385694
1,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,2578989,2,15,3,1.0,1.0,1,2,1.0,1.0,1.0,7.0,1.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,NaN,563277716385694
2,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,3312847,2,13,3,1.0,1.0,1,2,2.0,1.0,1.0,5.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563277716385694
3,4212908,42,4,1,2,268503,60.0,1.0,1.0,5.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,4,5502526707828,5128.0,102,0.0,10482075,2,38,1,1.0,2.0,1,2,1.0,3.0,NaN,NaN,NaN,4.0,4.0,1.0,1.0,NaN,1.0,6.0,1.0,12.0,563277716385694
4,2931905,29,2,2,3,2888440,0.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,4.0,1.0,3.0,3.0,1.0,3.0,2.0,2.0,0.0,2,550282046697327,2646.0,151,0.0,5553031,2,5,3,4.0,2.0,1,2,2.0,1.0,1.0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563263195326236


In [24]:
df.columns

Index(['cd_ibge', 'uf_ibge', 'regiao_ibge', 'estrato', 'classf', 'id_familia',
       'vlr_renda_media_fam', 'cod_local_domic_fam', 'cod_especie_domic_fam',
       'qtd_comodos_domic_fam', 'qtd_comodos_dormitorio_fam',
       'cod_material_piso_fam', 'cod_material_domic_fam',
       'cod_agua_canalizada_fam', 'cod_abaste_agua_domic_fam',
       'cod_banheiro_domic_fam', 'cod_escoa_sanitario_domic_fam',
       'cod_destino_lixo_domic_fam', 'cod_iluminacao_domic_fam',
       'cod_calcamento_domic_fam', 'cod_familia_indigena_fam',
       'ind_familia_quilombola_fam', 'ind_parc_mds_fam', 'qtde_pessoas',
       'peso.fam', 'dias_cadastramento', 'dias_atualizacao', 'classe_renda',
       'id_pessoa', 'cod_sexo_pessoa', 'idade', 'cod_parentesco_rf_pessoa',
       'cod_raca_cor_pessoa', 'cod_local_nascimento_pessoa',
       'cod_certidao_registrada_pessoa', 'cod_deficiencia_memb',
       'cod_sabe_ler_escrever_memb', 'ind_frequenta_escola_memb',
       'cod_escola_local_memb', 'cod_curso_frequ

In [25]:
# DELETANDO COLUNAS DESNECESSÁRIAS
print(f'Shape antes:', df.shape)
df.drop('cd_ibge', axis=1, inplace=True)
df.drop('regiao_ibge', axis=1, inplace=True)
df.drop('vlr_renda_media_fam', axis=1, inplace=True)
df.drop('peso.fam', axis=1, inplace=True)
df.drop('peso.pes', axis=1, inplace=True)
print(f'Shape depois:', df.shape)

Shape antes: (267140, 51)
Shape depois: (267140, 46)


In [26]:
# CRIANDO LISTA DE VARIÁVEIS POR TEMA PARA FACILITAR MANIPULAÇÃO E TRATAMENTO

var_id = ['id_familia', 'id_pessoa']
var_target = ['classe_renda']
var_trab = ['qtde_pessoas', 'idade', 'cod_trabalhou_memb', 'cod_afastado_trab_memb',
       'cod_agricultura_trab_memb', 'cod_principal_trab_memb',
       'cod_trabalho_12_meses_memb', 'qtd_meses_12_meses_memb']


## Tratando variáveis trabalho

In [27]:
# filtrando base
selected_columns = var_id + var_trab
df1 = df[selected_columns]

df1

,id_familia,id_pessoa,qtde_pessoas,idade,cod_trabalhou_memb,cod_afastado_trab_memb,cod_agricultura_trab_memb,cod_principal_trab_memb,cod_trabalho_12_meses_memb,qtd_meses_12_meses_memb
0,268503,2351897,4,4,NaN,NaN,NaN,NaN,NaN,NaN
1,268503,2578989,4,15,2.0,2.0,NaN,NaN,2.0,NaN
2,268503,3312847,4,13,NaN,NaN,NaN,NaN,NaN,NaN
3,268503,10482075,4,38,1.0,NaN,1.0,6.0,1.0,12.0
4,2888440,5553031,2,5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
267135,1481101,4367533,2,65,2.0,2.0,NaN,NaN,2.0,NaN
267136,1481101,8608500,2,53,2.0,2.0,NaN,NaN,2.0,NaN
267137,2934752,4274948,4,48,2.0,2.0,NaN,NaN,2.0,NaN
267138,2934752,6291827,4,15,2.0,2.0,NaN,NaN,2.0,NaN


In [28]:
df1.columns

Index(['id_familia', 'id_pessoa', 'qtde_pessoas', 'idade',
       'cod_trabalhou_memb', 'cod_afastado_trab_memb',
       'cod_agricultura_trab_memb', 'cod_principal_trab_memb',
       'cod_trabalho_12_meses_memb', 'qtd_meses_12_meses_memb'],
      dtype='object')

In [29]:
df1['cod_trabalho_12_meses_memb'].unique()
df1['cod_trabalhou_memb'].value_counts(normalize=True)

cod_trabalhou_memb
2.0    0.668631
1.0    0.331369
Name: proportion, dtype: float64

In [30]:
# Calcular percentual de pessoas por características de trabalho

df_var_trabalho = df1.groupby('id_familia').apply(
    lambda x: pd.Series({
        'pct_trabalho_semana_adulto': ((x['cod_trabalhou_memb'] == 1) & (x['idade'] >= 18) & (x['idade'] <= 59)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_trabalho_semana_idoso': ((x['cod_trabalhou_memb'] == 1) & (x['idade'] >= 60)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_trabalho_semana_crianca': ((x['cod_trabalhou_memb'] == 1) & (x['idade'] < 18)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_extrativista': ((x['cod_agricultura_trab_memb'] == 1) & (x['idade'] >= 18) & (x['idade'] <= 59)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_conta_propria': ((x['cod_principal_trab_memb'] == 1) & (x['idade'] >= 18) & (x['idade'] <= 59)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_trab_rural_temporario': ((x['cod_principal_trab_memb'] == 2) & (x['idade'] >= 18) & (x['idade'] <= 59)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_empregado_sem_carteira': ((x['cod_principal_trab_memb'] == 3) & (x['idade'] >= 18) & (x['idade'] <= 59)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_empregado_com_carteira': ((x['cod_principal_trab_memb'] == 4) & (x['idade'] >= 18) & (x['idade'] <= 59)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_trab_domestico_sem_carteira': ((x['cod_principal_trab_memb'] == 5) & (x['idade'] >= 18) & (x['idade'] <= 59)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_trab_nao_remunerado': ((x['cod_principal_trab_memb'] == 7) & (x['idade'] >= 18) & (x['idade'] <= 59)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_militar_servidor': ((x['cod_principal_trab_memb'] == 8) & (x['idade'] >= 18) & (x['idade'] <= 59)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_trabalho_12meses_adulto': ((x['cod_trabalho_12_meses_memb'] == 1) & (x['idade'] >= 18) & (x['idade'] <= 59)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_trabalho_12meses_idoso': ((x['cod_trabalho_12_meses_memb'] == 1) & (x['idade'] >= 60)).sum() / x['qtde_pessoas'].iloc[0],
        'pct_trabalho_12meses_crianca': ((x['cod_trabalho_12_meses_memb'] == 1) & (x['idade'] < 18)).sum() / x['qtde_pessoas'].iloc[0],
    }),
    include_groups=False
).reset_index()


In [31]:
# ANALISANDO DADOS FALTANTES
missing_data_info(df_var_trabalho)

,Missing Count,Missing Percent,Data Type
id_familia,0,0.0,int64
pct_trabalho_semana_adulto,0,0.0,float64
pct_trabalho_semana_idoso,0,0.0,float64
pct_trabalho_semana_crianca,0,0.0,float64
pct_extrativista,0,0.0,float64
pct_conta_propria,0,0.0,float64
pct_trab_rural_temporario,0,0.0,float64
pct_empregado_sem_carteira,0,0.0,float64
pct_empregado_com_carteira,0,0.0,float64
pct_trab_domestico_sem_carteira,0,0.0,float64


In [32]:

# Salvando base
df_var_trabalho.to_csv('base_pre/df_var_trabalho.csv', index=False)